<h2> Brady Kruse & Marcus Jordan </h2>

<h1> Nim Rules (for reference) </h1>

<h1> Nim Rules & Tree Setup </h1>

In [190]:
def childrenPopulate(nim, depth):
    children = []
    stack_index = 0
    for each in nim:
        for i in range(1, each.size+1):
            temp_nim = copy.deepcopy(nim)
            temp_nim[stack_index].removeStone(i)
            children.append(Node(temp_nim, depth, childrenPopulate(temp_nim, depth+1)))
        stack_index += 1
    return children

def firstDepthChildren(nim, player):
    children = []
    stack_index = 0
    for stack in nim:
        for i in range(1, stack.size+1):
            temp_nim = copy.deepcopy(nim)
            temp_nim[stack_index].removeStone(i)
            children.append(Node(temp_nim, 1, player=not player))
        stack_index += 1
    return children

In [191]:
def playerSet(node, role):
    node.player = role
    for child in node.children:
        playerSet(child, not node.player)

In [192]:
count = 0
def printAllNodes(node):
    global count
    children = node.children
    for child in children:
        printAllNodes(child)
    
    node.printNode()
    count += 1

In [193]:
class Stack:
    def __init__(self, size):
        self.size = size
        self.original_size = size
    
    def removeStone(self, amount):
        if amount <= self.size and amount != 0:
            self.size -= amount
            return True #Indicates a valid turn
        
        return False #Indicates an invalid turn

In [194]:
class Node: #For ease, TRUE is the MAX player and FALSE is the MIN player
    def __init__(self, state, depth, children=None, parent=None, player=None):
        self.state = state
        self.parent = parent
        self.children = children
        self.depth = depth
        self.player = None
        
        if children != None:
            for child in self.children:
                child.parent = self
    
    def printNode(self): #Have no idea why, on Ubuntu end = "" throws a syntax error? Something to do w/ how printing is handled
        print("Node: ")
        for stack in self.state: print(stack.size)#, end=" ")
        print("\n")
    
        print("Player: ")#, end ="")
        if (self.player):
            print("AI\n")
        else:
            print("PUNY HUMAN\n")
        
        print("Depth: ", self.depth)

        if(self.parent != None):
            print("Parent: ")
            for stack in self.parent.state: print(stack.size)#, end = " ")
            print("\n")

        if(self.children != None):
            print("Children: ")
            for child in self.children:
                for stack in child.state: print(stack.size)#, end = " ")
                print("\n")
        
        print("--------------")
    def validTurn(self): #Check each stack, if all are 0, no valid turns are left & Return false. 
        dead_stacks = 0
        for stack in self.state:
            if stack.size == 0:
                dead_stacks+=1
        if(dead_stacks == len(self.state)):
            return False
        return True

In [195]:
class NimTree:
    def __init__(self, nim, player):
        self.nim = nim
        self.origin = Node(nim, 0)
        self.origin.children = childrenPopulate(self.origin.state, self.origin.depth)
        self.win_count = 0
        self.win_depth = 100
        self.win_node = None
        playerSet(self.origin, player)
        
    def possibleWins(self, node):
        for child in node.children:
            self.possibleWins(child)
            
        if not node.validTurn() and not node.player:
            #print("WIN")
            #node.printNode()
            #print("--- turn end ---" + "\n")
            self.win_count += 1
        elif not node.validTurn() and node.player:
            #print("LOSS")
            #node.printNode()
            #print("--- turn end ---" + "\n")
            return
        else:
            return
            
    def countWins(self): 
        self.possibleWins(self.origin)
        return self.win_count
    
    def lowestWinDepth(self, node):
        for child in node.children:
            self.lowestWinDepth(child)
                
        if not node.validTurn() and not node.player:
            #print("WIN")
            #node.printNode()
            #print("--- turn end ---" + "\n")
            if node.depth < self.win_depth:
                self.win_node = node
                self.win_depth = node.depth
        elif not node.validTurn() and node.player:
            #print("LOSS")
            #node.printNode()
            #print("--- turn end ---" + "\n")
            return
        else:
            return
        

In [196]:
import copy
import math

In [197]:
nim = [Stack(3), Stack(3), Stack(3)]

In [198]:
for stack in nim: print(stack.size)

3
3
3


In [199]:
nim = [Stack(3), Stack(3), Stack(3)]
tree = NimTree(nim, False) 

In [200]:
#this prints the nim state of the origin node of the tree
for stack in tree.origin.state: print(stack.size)#, end = " ")
print("\n") 

3
3
3




In [201]:
#This prints the nim state of all of the children nodes of a node (in this case, origin)
for child in tree.origin.children:
    for stack in child.state: print(stack.size)#, end = " ")
    print("\n")

2
3
3


1
3
3


0
3
3


3
2
3


3
1
3


3
0
3


3
3
2


3
3
1


3
3
0




In [203]:
#Commented out because this takes 5ever to run
#We're getting some repeated nodes...I think this is how it should be though?
#count = 0
#printAllNodes(tree.origin)

#print(count)

In [204]:
custom_nim = [Stack(0), Stack(0), Stack(3)]
custom_tree = NimTree(custom_nim, True)
printAllNodes(custom_tree.origin)

Node: 
0
0
0


Player: 
PUNY HUMAN

Depth:  2
Parent: 
0
0
1


Children: 
--------------
Node: 
0
0
1


Player: 
AI

Depth:  1
Parent: 
0
0
2


Children: 
0
0
0


--------------
Node: 
0
0
0


Player: 
AI

Depth:  1
Parent: 
0
0
2


Children: 
--------------
Node: 
0
0
2


Player: 
PUNY HUMAN

Depth:  0
Children: 
0
0
1


0
0
0


--------------
Node: 
0
0
0


Player: 
AI

Depth:  1
Parent: 
0
0
1


Children: 
--------------
Node: 
0
0
1


Player: 
PUNY HUMAN

Depth:  0
Children: 
0
0
0


--------------
Node: 
0
0
0


Player: 
PUNY HUMAN

Depth:  0
Children: 
--------------
Node: 
0
0
3


Player: 
AI

Depth:  0
Children: 
0
0
2


0
0
1


0
0
0


--------------


In [205]:
print(custom_tree.countWins())

2


<h1> Minimax Algorithm </h1>

In [206]:
def minimaxAI(node):
    node_win_count = 0
    next_move = None
    node_children = firstDepthChildren(node.state, node.player)
    for child in node_children:
        #child.printNode()
        child_tree = NimTree(child.state, child.player)
        child_wins = child_tree.countWins()
        if child_wins > node_win_count:
            node_win_count = child_wins
            next_move = child
    
    return node_win_count, next_move
    

In [207]:
custom_nim = [Stack(0), Stack(0), Stack(3)]
custom_nim_node = Node(custom_nim, depth=0, player=True)
AI_wins, AI_next_move = minimaxAI(custom_nim_node)
print("AI Wins: ", AI_wins, "\n\nNext Move:\n")
AI_next_move.printNode()

AI Wins:  1 

Next Move:

Node: 
0
0
2


Player: 
PUNY HUMAN

Depth:  1
--------------


In [208]:
def gameLoopMinimax(nim, player):
    print("\t   GAME START\nMAY NIMROD HAVE MERCY ON YOUR SOUL\n----------------------------------\n")
    current_state = Node(nim, 0)
    current_state.player = player
    current_state.printNode()
    print("----------------------------------\n")
    
    while(True):
        if(current_state.player): #AI's Turn
            AI_wins, AI_next_move = minimaxAI(current_state)
            print(AI_wins)
            if(AI_wins == 0):
                print("You Won! Somehow...")
                break
                
            if(not AI_next_move.validTurn()):
                current_state = Node([Stack(0), Stack(0), Stack(0)], 0)
                print("AI Moves To")
                current_state.printNode()
                print("THE MIGHTY NIMROD BESTED YOU AGAIN")
                break
    
            current_state = AI_next_move
            
            print("AI Moves To:")
            current_state.printNode()
            print("----------------END TURN------------------\n")
            
        else: #Your turn!
            print("It's Your Turn")
            for stack in current_state.state:
                print("Stack: ", current_state.state.index(stack), ": ", stack.size)
            
            stack_choice = int(input("Stack: "))
            remove_amount = int(input("Stone #: "))
            while(stack_choice > len(current_state.state)-1):
                stack_choice = int(input("Pick Valid Stack: "))
                remove_amount = int(input("Stone #: "))
            while(not current_state.state[stack_choice].removeStone(remove_amount)):
                print("INVALID CHOICE! THIS ANGERS NIMROD!")
                stack_choice = int(input("Pick Valid Stack: "))
                remove_amount = int(input("Stone #: "))
            
            print("\nYou Move To:")
            current_state.player = not current_state.player
            current_state.printNode()
            print("----------------END TURN------------------\n")

In [ ]:
test_nim = [Stack(0), Stack(2), Stack(3)]
gameLoopMinimax(test_nim, False)

<h1>Depth Heuristic w/ AB Pruning</h1>

In [209]:
class ImprovedNode: #For ease, TRUE is the MAX player and FALSE is the MIN player
    def __init__(self, state, depth, children=None, parent=None, player=None):
        self.state = state
        self.parent = parent
        self.children = children
        self.depth = depth
        self.player = player
        
        if children != None:
            for child in self.children:
                child.parent = self
    
    def printNode(self): #Have no idea why, on Ubuntu end = "" throws a syntax error? Something to do w/ how printing is handled
        print("Node: ")
        for stack in self.state: print(stack.size)#, end=" ")
        print("\n")
    
        print("Player: ")#, end ="")
        if (self.player):
            print("AI\n")
        else:
            print("PUNY HUMAN\n")
        
        print("Depth: ", self.depth)

        if(self.parent != None):
            print("Parent: ")
            for stack in self.parent.state: print(stack.size)#, end = " ")
            print("\n")

        if(self.children != None):
            print("Children: ")
            for child in self.children:
                for stack in child.state: print(stack.size)#, end = " ")
                print("\n")
        
        print("--------------")
    def validTurn(self): #Check each stack, if all are 0, no valid turns are left & Return false. 
        dead_stacks = 0
        for stack in self.state:
            if stack.size == 0:
                dead_stacks+=1
        if(dead_stacks == len(self.state)):
            return False
        return True

In [210]:
def isWinState(node):
    if not node.validTurn() and not node.player:
        return True

In [211]:
def firstDepthChildrenImproved(nim, player, depth):
    children = []
    stack_index = 0
    for stack in nim:
        for i in range(1, stack.size+1):
            temp_nim = copy.deepcopy(nim)
            temp_nim[stack_index].removeStone(i)
            children.append(ImprovedNode(temp_nim, depth+1, player= not player))
        stack_index += 1
    return children

In [212]:
def depthFirstSearchABPrune(node):
    global current_win_depth
    node_children = firstDepthChildrenImproved(node.state, node.player, node.depth)
    for child in node_children:
        if child.depth >= current_win_depth:
            return
        elif isWinState(child) and child.depth < current_win_depth:
            current_win_depth = child.depth
        else:
            depthFirstSearchABPrune(child)
    return current_win_depth

In [213]:
current_win_depth = 50
custom_nim = [Stack(0), Stack(2), Stack(2)]
custom_nim_node = ImprovedNode(custom_nim, depth=0, player=True)
depthFirstSearchABPrune(custom_nim_node)

3

In [214]:
def depthAI(node):
    next_move = None
    win_depth = 100
    node_children = firstDepthChildrenImproved(node.state, node.player, node.depth)
    for child in node_children:
        child_win_depth = depthFirstSearchABPrune(child)
        if child_win_depth != None:
            if child_win_depth < win_depth:
                win_depth = child_win_depth
                next_move = child
    return next_move

In [215]:
def gameLoopDepth(nim, player):
    print("\t   GAME START\nMAY NIMROD HAVE MERCY ON YOUR SOUL\n----------------------------------\n")
    current_state = Node(nim, 0)
    current_state.player = player
    current_state.printNode()
    print("----------------------------------\n")
    
    while(True):
        if(current_state.player): #AI's Turn
            AI_next_move = depthAI(current_state)
            if(AI_next_move == None):
                print("You Won! Somehow...")
                break
                
            if(not AI_next_move.validTurn()):
                current_state = Node([Stack(0), Stack(0), Stack(0)], 0)
                print("AI Moves To")
                current_state.printNode()
                print("THE MIGHTY NIMROD BESTED YOU AGAIN")
                break
    
            current_state = AI_next_move
            
            print("AI Moves To:")
            current_state.printNode()
            print("----------------END TURN------------------\n")
            
        else: #Your turn!
            print("It's Your Turn")
            for stack in current_state.state:
                print("Stack: ", current_state.state.index(stack), ": ", stack.size)
            
            stack_choice = int(input("Stack: "))
            remove_amount = int(input("Stone #: "))
            while(stack_choice > len(current_state.state)-1):
                stack_choice = int(input("Pick Valid Stack: "))
                remove_amount = int(input("Stone #: "))
            while(not current_state.state[stack_choice].removeStone(remove_amount)):
                print("INVALID CHOICE! THIS ANGERS NIMROD!")
                stack_choice = int(input("Pick Valid Stack: "))
                remove_amount = int(input("Stone #: "))
            
            print("\nYou Move To:")
            current_state.player = not current_state.player
            current_state.printNode()
            print("----------------END TURN------------------\n")

In [ ]:
test_nim = [Stack(3), Stack(3), Stack(3)]
gameLoopDepth(test_nim, False)

<h1> Perfect Binary Static Heuristic </h1>

In [221]:
def binarySplit(num):
    return list(str(bin(num))[2:])

In [222]:
def binaryHeuristic(node):
    binary_list = []
    length = 0
    for stack in node.state:
        binary_list.append(binarySplit(stack.size))
    for item in binary_list:
        if len(item) > length:
            length = len(item)
    for item in binary_list:
        if len(item) < length:
            for i in range(length-len(item)):
                item.insert(0,'0')
    return binary_list

In [223]:
def binaryDigitalSum(binary_list):
    sum = []
    for i in range(len(binary_list[0])):
        sum.append(0)
    for each in binary_list:
        for i in range(len(each)):
            sum[i] += int(each[i])
    return sum

In [224]:
def allEven(binary_list):
    for num in binary_list:
        if num % 2 != 0:
            return False
    return True

In [225]:
custom_nim = [Stack(0), Stack(2), Stack(2)]
custom_nim_node = Node(custom_nim, depth=0, player=True)
binaryHeuristic(custom_nim_node)
allEven(binaryDigitalSum(binaryHeuristic(custom_nim_node)))

True

In [226]:
def perfectAI(node): #Currently: no depth calculation
    node_children = firstDepthChildren(node.state, node.player)
    for each in node_children:
        binary_sum = binaryDigitalSum(binaryHeuristic(each))
        if(allEven(binary_sum)):
            next_move = each
            next_move.printNode()
            return next_move
        
    node_win_count = 0 #If already in a losing state, go whichever path has the most wins. There's a more elegant solution for this
    for child in node_children:
        #child.printNode()
        child_tree = NimTree(child.state, child.player)
        child_wins = child_tree.countWins()
        if child_wins > node_win_count:
            node_win_count = child_wins
            next_move = child
    return next_move

In [227]:
custom_nim = [Stack(0), Stack(2), Stack(3)]
custom_nim_node = Node(custom_nim, depth=0, player=True)
test_node = perfectAI(custom_nim_node)
test_node.printNode()

Node: 
0
2
2


Player: 
PUNY HUMAN

Depth:  1
--------------
Node: 
0
2
2


Player: 
PUNY HUMAN

Depth:  1
--------------


In [228]:
def gameLoopStatic(nim, player):
    print("\t   GAME START\nMAY NIMROD HAVE MERCY ON YOUR SOUL\n----------------------------------\n")
    current_state = Node(nim, 0)
    current_state.player = player
    current_state.printNode()
    print("----------------------------------\n")
    
    while(True):
        if(current_state.player): #AI's Turn
            AI_next_move = perfectAI(current_state)
            if(AI_next_move == None):
                print("You Won! Somehow...")
                break
                
            if(not AI_next_move.validTurn()):
                current_state = Node([Stack(0), Stack(0), Stack(0)], 0)
                print("AI Moves To")
                current_state.printNode()
                print("THE MIGHTY NIMROD BESTED YOU AGAIN")
                break
    
            current_state = AI_next_move
            
            print("AI Moves To:")
            current_state.printNode()
            print("----------------END TURN------------------\n")
            
        else: #Your turn!
            print("It's Your Turn")
            for stack in current_state.state:
                print("Stack: ", current_state.state.index(stack), ": ", stack.size)
            
            stack_choice = int(input("Stack: "))
            remove_amount = int(input("Stone #: "))
            while(stack_choice > len(current_state.state)-1):
                stack_choice = int(input("Pick Valid Stack: "))
                remove_amount = int(input("Stone #: "))
            while(not current_state.state[stack_choice].removeStone(remove_amount)):
                print("INVALID CHOICE! THIS ANGERS NIMROD!")
                stack_choice = int(input("Pick Valid Stack: "))
                remove_amount = int(input("Stone #: "))
            
            print("\nYou Move To:")
            current_state.player = not current_state.player
            current_state.printNode()
            print("----------------END TURN------------------\n")

test_nim = [Stack(0), Stack(2), Stack(3)]
gameLoopStatic(test_nim, True)

<h1> Try Them All! </h1>

In [233]:
test_nim = [Stack(1), Stack(2), Stack(1)]
gameLoopMinimax(test_nim, True) #AI goes first

	   GAME START
MAY NIMROD HAVE MERCY ON YOUR SOUL
----------------------------------

Node: 
1
2
1


Player: 
AI

Depth:  0
--------------
----------------------------------

2
AI Moves To:
Node: 
0
2
1


Player: 
PUNY HUMAN

Depth:  1
--------------
----------------END TURN------------------

It's Your Turn
Stack:  0 :  0
Stack:  1 :  2
Stack:  2 :  1
Stack: 2
Stone #: 1

You Move To:
Node: 
0
2
0


Player: 
AI

Depth:  1
--------------
----------------END TURN------------------

1
AI Moves To
Node: 
0
0
0


Player: 
PUNY HUMAN

Depth:  0
--------------
THE MIGHTY NIMROD BESTED YOU AGAIN


In [234]:
test_nim = [Stack(1), Stack(2), Stack(1)]
gameLoopDepth(test_nim, True)

	   GAME START
MAY NIMROD HAVE MERCY ON YOUR SOUL
----------------------------------

Node: 
1
2
1


Player: 
AI

Depth:  0
--------------
----------------------------------

AI Moves To:
Node: 
0
2
1


Player: 
PUNY HUMAN

Depth:  1
--------------
----------------END TURN------------------

It's Your Turn
Stack:  0 :  0
Stack:  1 :  2
Stack:  2 :  1
Stack: 1
Stone #: 2

You Move To:
Node: 
0
0
1


Player: 
AI

Depth:  1
--------------
----------------END TURN------------------

AI Moves To
Node: 
0
0
0


Player: 
PUNY HUMAN

Depth:  0
--------------
THE MIGHTY NIMROD BESTED YOU AGAIN


In [235]:
test_nim = [Stack(1), Stack(2), Stack(1)]
gameLoopStatic(test_nim, True)

	   GAME START
MAY NIMROD HAVE MERCY ON YOUR SOUL
----------------------------------

Node: 
1
2
1


Player: 
AI

Depth:  0
--------------
----------------------------------

Node: 
1
0
1


Player: 
PUNY HUMAN

Depth:  1
--------------
AI Moves To:
Node: 
1
0
1


Player: 
PUNY HUMAN

Depth:  1
--------------
----------------END TURN------------------

It's Your Turn
Stack:  0 :  1
Stack:  1 :  0
Stack:  2 :  1
Stack: 0
Stone #: 1

You Move To:
Node: 
0
0
1


Player: 
AI

Depth:  1
--------------
----------------END TURN------------------

Node: 
0
0
0


Player: 
PUNY HUMAN

Depth:  1
--------------
AI Moves To
Node: 
0
0
0


Player: 
PUNY HUMAN

Depth:  0
--------------
THE MIGHTY NIMROD BESTED YOU AGAIN


<h1> Closing Thoughts </h1>

There's certainly PLENTY of polish that can be added to this project. The code can certainly be cleaner (there's several instances of redundancy and inconsistency withiin certain classes), organized better (test code is currently scattered throughout), and several things that can be done to the ACTUAL AI. A "perfect" heuristic would probably be some combination of the three (due to the random nature of human), although that would take plenty of tinkering around with different levels of depth. For example, a heuristic only chasing wins will behave stupidly at times, but a depth-heuristic might pigeonhole itself down a very specific scenario that has an easy win. The binary static heuristic is mathematically the best, but when stuck in a losing situation, there is most certainly a more elegant way to pick the "best" path than just counting the number of wins. A much better solution would be some combination of quickly arriving at an odd scenario while also leaving flexibility for multiple odd scenarios. As far as speed--this algorithm is painfully slow, although it does a FANTASTIC job of hiding. Simply put, humans are stupid: generating trees relies on creating EVERY possibility (down to different roles, parents, etc.) instead of simply the "logical" ones. Also, since the "wins" rely on generating the FINAL leaf, alpha-beta pruning is impossible to implement since we MUST generate all the way down anyway. What I'd really like to do is put the AI's against each other in a best of 100 tournament (or at least until mathematical equilibrium is hit), but that will take a significant amount of coding I'm afraid. Also there is certainly ability for game theory to come into play here.

As always, improvement is infinite. Nonetheless, I am very excited about beginning my first foray into AI. And so we beat on, boats against the current, always advancing despite the past. 